In [ ]:
from fastembed import TextEmbedding
import numpy as np

documents: list[str] = [
    "passage: Hello, World!",
    "query: Hello, World!",
    "passage: This is an example passage.",
    "fastembed is supported by and maintained by Qdrant."
]

#embedding_model = TextEmbedding() #
#embeddings: list[np.ndarray] = embedding_model.embed(documents) #


""" 
Embedding in batches
Small batch (8–16) → Lower memory use, but slower (more runtime calls).
Medium batch (32–128) → Often the sweet spot on CPU.
Large batch (256+) → Can be faster on GPUs or beefy CPUs, but risks OOM (out of memory) errors.
"""
embedding_model = TextEmbedding(model_name="BAAI/bge-small-en-v1.5") # change model here : MiniLM
embeddings = list(embedding_model.embed(documents, batch_size=64))
print(f"Got {len(embeddings)} embeddings")




Got 4 embeddings


In [2]:
!pip install fastembed

  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached onnxruntime-1.22.1-cp313-cp313-macosx_13_0_universal2.whl.metadata (4.6 kB)
  Using cached pillow-11.3.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached hf_xet-1.1.10-cp37-abi3-macosx_11_0_arm64.whl.metadata (4.7 kB)
  Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using ca

In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")  # Using an in-process Qdrant
## CAN BUILD CACHE

docs = ["Qdrant has Langchain integrations", "Qdrant also has Llama Index integrations"]
metadata = [
    {"source": "Langchain-docs"},
    {"source": "Llama-index-docs"},
]
ids = [42, 2]

client.add(
    collection_name="demo_collection",
    documents=docs,
    metadata=metadata,
    ids=ids
)

search_result = client.query(
    collection_name="demo_collection",
    query_text="This is a query document"
)
print(search_result)